## Address missing data, crop, detrend/normalize, and save timeseries netcdf

Function of script:
* Remove days with too few hours and years with too few days
* Crop (e.g., ater 1960)
* Update metadata record length
* Detrend based on linear slope
* Normalize (shift) to base period mean (e.g., 1995-2014) 
* Format and save as netcdf

Context in workflow:
* Must be run after preproc
* stn_thresh and peaks-per-yr calculated separately

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import xarray as xr
from pathlib import Path

%matplotlib inline

base_start = 1995 
base_end = 2014
missing_value = 999.999
dec = 3
max_missing = 0.3
min_hrs_per_day = 12
decluster = 25 # hours
start_crop = '1960'

DATA = Path("data")
INPUTS = DATA / "inputs"
OUTPUTS = DATA / "outputs"

stnlist_csv = INPUTS / "metadata.csv"
stnlist_csv_edit = OUTPUTS / "metadata.csv"
input_dir = OUTPUTS / "wl_preproc"
output_dir = OUTPUTS / "wl_proc"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
stnlist = pd.read_csv(stnlist_csv, encoding='latin1')

In [ ]:
for i, row in stnlist.iterrows():
    stn_num = str(row['stn_num']).zfill(5)
    stn_name = row['stn_name']
    datapath_in = os.path.join(input_dir, f"{stn_num}_HOURLY.csv") 
    wl_datapath_out = os.path.join(output_dir, f"{stn_num}_wl.nc") 
    wl_pot_datapath_out = os.path.join(output_dir, f"{stn_num}_wl_pot.nc") 
    print(stn_num, stn_name)
    
    df = pd.read_csv(datapath_in, encoding='latin-1', parse_dates=['DateTime_utc'], index_col='DateTime_utc')
    df['wl_CGVD2013'] = df['wl_CGVD2013'].replace(missing_value, np.nan)
    df = df.dropna()

    # Remove days with too few hours 
    if not df.dropna().empty:
        hrs_per_day = df.resample('D').count()
        boolean_bad_days = hrs_per_day < min_hrs_per_day
        bad_days = boolean_bad_days[boolean_bad_days].index
        df = df[~df.index.isin(bad_days)]

    # Remove years with too few days
    if not df.dropna().empty:
        days_per_year = df.dropna().resample('YE').apply(lambda x: pd.Index(x.index.date).nunique())
        boolean_bad_yrs = days_per_year < (365.25*(1-max_missing))
        bad_yrs = boolean_bad_yrs[boolean_bad_yrs['wl_CGVD2013']==True].index.year
        df = df[~df.index.year.isin(bad_yrs)]
        
    # Crop after 1960
    if not df.empty:
        df = df.loc[df.index >= pd.to_datetime(start_crop).tz_localize('UTC')]
        crop_nyrs_based_on_hrs = round(df['wl_CGVD2013'].dropna().count()/(365.25*24), 3)
        crop_calendar_nyrs = len(df.dropna().index.year.unique())
        crop_start_yr = int(df.index[0].year)
        crop_end_yr = int(df.index[-1].year)
        
        if crop_nyrs_based_on_hrs > crop_calendar_nyrs:
            raise ValueError("nyrs_based_on_hrs should be less than or equal to calendar_nyrs")
    else:
        crop_nyrs_based_on_hrs, crop_calendar_nyrs, crop_start_yr, crop_end_yr = None, None, None, None
    
    # Update metadata after cropping
    stnlist.at[i, 'calendar_nyrs'] = crop_calendar_nyrs
    stnlist.at[i, 'nyrs_based_on_hrs'] = crop_nyrs_based_on_hrs
    stnlist.at[i, 'start_yr'] = crop_start_yr
    stnlist.at[i, 'end_yr'] = crop_end_yr
    
    # Detrend
    df_yearly = df['wl_CGVD2013'].resample('YE').mean()
    df_yearly = df_yearly.dropna()
    time_ordinal_yearly = df_yearly.index.to_series().apply(lambda x: x.toordinal())
    slope_per_day, intercept = np.polyfit(time_ordinal_yearly, df_yearly.values, 1)
    time_ordinal_hourly = df.index.to_series().apply(lambda x: x.toordinal())
    yearly_detrend_hourly_points = slope_per_day * time_ordinal_hourly + intercept
    df["wl_CGVD2013_detrend"] = df["wl_CGVD2013"] - yearly_detrend_hourly_points
    sl_m_per_yr = slope_per_day * 365.25 
    sl_mm_per_yr = round(sl_m_per_yr * 1000, dec) 
    stnlist.at[i, 'sl_mm_per_yr'] = sl_mm_per_yr
    
    # Normalize
    def mid_yr(base_start, base_end):
        middle_year = (base_end - base_start) / 2 + base_start
        year = int(middle_year)
        fraction = middle_year - year
        days = int(fraction * 365.25)
        date = datetime(year, 1, 1) + pd.Timedelta(days=days)
        return date
    mid_yr_ordinal = mid_yr(base_start, base_end).toordinal()
    residual_mean_after_detrend = df['wl_CGVD2013_detrend'].mean()
    period_mean = slope_per_day*mid_yr_ordinal + intercept
    df['wl_CGVD2013_detrend_norm'] = df['wl_CGVD2013_detrend'] - residual_mean_after_detrend + period_mean
    
    # Format & save netcdf file
    df1 = df['wl_CGVD2013_detrend_norm'].reset_index()
    df1.columns = ['time', 'wl']
    wl = xr.DataArray(df1['wl'], dims=['time'], coords={'time': df1['time']}) #.rename("wl")
        
    attrs = {
        "wl_stn_name": stn_name,
        "wl_stn_id": stn_num,
        "units": "m",
        "standard_name": "sea_surface_height_above_geopotential_datum",
        "geopotential_datum_name": "Canadian Geodetic Vertical Datum of 2013 (CGVD2013)",
        "sl_mm_yr": sl_mm_per_yr,
        "ref_period": [base_start, base_end],
        "label": "Measured Water Level Timeseries",
        "calendar_nyrs": crop_calendar_nyrs,
        "intercept_ordinal": intercept,
    }
    
    with xr.set_options(keep_attrs=True):
        wl.attrs.update(attrs)
    wl.to_netcdf(wl_datapath_out)
    
stnlist.to_csv(stnlist_csv_edit, index=False, encoding='latin1')